In [3]:
def install_dependencies():
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", "xformers", "trl<0.9.0", "peft", "accelerate", "bitsandbytes"])

def load_packages():
    global FastLanguageModel, pd, Dataset, SFTTrainer, TrainingArguments, is_bfloat16_supported
    from unsloth import FastLanguageModel
    import pandas as pd
    from datasets import Dataset
    from trl import SFTTrainer
    from transformers import TrainingArguments
    from unsloth import is_bfloat16_supported

def load_model(model_name, max_seq_length=2048, dtype=None, load_in_4bit=True):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit
    )
    return model, tokenizer

def configure_model(model, r=16, target_modules=None, lora_alpha=16, lora_dropout=0, bias="none", use_gradient_checkpointing="unsloth", random_state=3407, use_rslora=False, loftq_config=None):
    if target_modules is None:
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    model = FastLanguageModel.get_peft_model(
        model,
        r=r,
        target_modules=target_modules,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias=bias,
        use_gradient_checkpointing=use_gradient_checkpointing,
        random_state=random_state,
        use_rslora=use_rslora,
        loftq_config=loftq_config,
    )
    return model

def prepare_dataset(prompt, tokenizer):
    df = pd.read_excel("train.xlsx")
    EOS_TOKEN = tokenizer.eos_token
    df["question"] = (
        prompt
        + "\n Question: )"
        + df["Question"]
        + "\n Code: )"
        + df["Code"]
        + "\n Plain Text: )"
        + df["Plain Text"]
        + EOS_TOKEN
    )

    custom_ds = pd.DataFrame()
    custom_ds["text"] = df["question"]
    dataset = Dataset.from_pandas(custom_ds)
    return dataset

def train_model(model, tokenizer, dataset, output_dir="outputs"):

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        max_seq_length=2048,
        dataset_num_proc=2,
        packing=False,
        args=TrainingArguments(
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=5,
            max_steps=15,
            learning_rate=2e-4,
            fp16=not is_bfloat16_supported(),
            bf16=is_bfloat16_supported(),
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            output_dir=output_dir,
        ),
    )
    trainer_stats = trainer.train()
    return trainer_stats

def generate_response(model, tokenizer, prompt):

    test_df = pd.read_excel("test.xlsx")
    test_row = test_df.tail(1)
    test_text = prompt + "\n Question: )" + test_row["Question"].values[0] + "\n Code: )" + test_row["Code"].values[0] + "\n Plain Text: )"
    print(test_text)

    FastLanguageModel.for_inference(model)

    inputs = tokenizer([test_text], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=200)

    # Decode the generated tokens, excluding the input tokens
    generated_sequence = outputs[0]
    input_length = inputs['input_ids'].shape[1]  # Length of the input text tokens
    new_tokens = generated_sequence[input_length:]  # Exclude the input tokens

    new_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return new_text

In [5]:
def main():
    install_dependencies()
    load_packages()

    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    # model_name = 'unsloth/llama-3-8b-Instruct-bnb-4bit',
    # model_name = 'unsloth/mistral-7b-instruct-v0.3-bnb-4bit',
    model_name = 'unsloth/mistral-7b-v0.3-bnb-4bit'

    model, tokenizer = load_model(model_name)
    model = configure_model(model)

    prompt = "Summarize the provided code solution for the given problem in simple, plain English. Explain step-by-step how the code works to solve the problem in a way that is easy to understand."
    dataset = prepare_dataset(prompt,tokenizer)

    trainer_stats = train_model(model, tokenizer, dataset)

    response = generate_response(model, tokenizer, prompt)
    print(response)

if __name__ == "__main__":
    main()

==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/20 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 15
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.978200
2,0.862500
3,1.007600
4,0.702000
5,0.696200
6,0.542700
7,0.431100
8,0.357900
9,0.341000
10,0.254800


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summarize the provided code solution for the given problem in simple, plain English. Explain step-by-step how the code works to solve the problem in a way that is easy to understand.
 Question: )Given a string s containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

An input string is valid if:

Open brackets must be closed by the same type of brackets.
Open brackets must be closed in the correct order.
Every close bracket has a corresponding open bracket of the same type.
 

Example 1:

Input: s = "()"
Output: true
Example 2:

Input: s = "()[]{}"
Output: true
 Code: )class Solution:
    def isValid(self, s: str) -> bool:
        Map = {")": "(", "]": "[", "}": "{"}
        stack = []

        for c in s:
            if c not in Map:
                stack.append(c)
                continue
            if not stack or stack[-1] != Map[c]:
                return False
            stack.pop()

        return not stack

 Plain Text: )
Use a